In [1]:
%%time

import time 
import timeit 

start_time = time.time()

# the prime for the secp256k1 is 2^256 - 2^32 - 977 
p = 115792089237316195423570985008687907853269984665640564039457584007908834671663

# Bob's private key that we plan to find out using the attack 
b1 = 0xc3549423
b2 = 0xF837BC8C4461CCD7
b3 = 0x6a9d9fd679c147a78cbc6857dec02e76
b = b3

# the secp256k1 curve over the finite field Z_p
curve = EllipticCurve(GF(p), [0, 7])

print(p)

115792089237316195423570985008687907853269984665640564039457584007908834671663
CPU times: user 72.7 ms, sys: 0 ns, total: 72.7 ms
Wall time: 72.1 ms


In [2]:
%%time
# function for generator and order 
def gen_order(curve):

    grp = curve.abelian_group()
    gen = grp.gens()[0]

    group_order = gen.order()
    # print("{0} = {1}".format(group_order, factor(group_order)))

    return (gen, group_order)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.58 µs


In [3]:
%%time

# function for generator for the "small" subgroup 
def gen_subgroup(P, group_order, subgroup_order): 
    
    temp = group_order // subgroup_order 
    x = crt([1,0], [subgroup_order, temp])
    
    return (x*P)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.58 µs


In [4]:
%%time

# simple encryption 
def encrypt(symmkey, message):
    return symmkey

def key_does_decrypt(symmkey, cipherText):
    return (symmkey == cipherText)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


In [5]:
%%time

twist_1 = EllipticCurve(GF(p), [0, 1])
twist_2 = EllipticCurve(GF(p), [0, 2])
twist_3 = EllipticCurve(GF(p), [0, 3])
twist_4 = EllipticCurve(GF(p), [0, 4])
twist_6 = EllipticCurve(GF(p), [0, 6])

# small subgroup for twist_1 is of order 20412485227
(gen, order) = gen_order(twist_1)
s1 = [20412485227]
P11 = gen_subgroup(gen, order, s1[0])

# small subgroups for twist_2 are of order 3319, 22639
(gen, order) = gen_order(twist_2)
s2 = [3319, 22639]
P21 = gen_subgroup(gen, order, s2[0])
P22 = gen_subgroup(gen, order, s2[1])

# small subgroups for twist_3 are of order 109903, 12977017, 383229727
(gen, order) = gen_order(twist_3)
s3 = [109903, 12977017, 383229727] 
P31 = gen_subgroup(gen, order, s3[0])
P32 = gen_subgroup(gen, order, s3[1])
P33 = gen_subgroup(gen, order, s3[2])

# small subgroup for twist_4 is of order 18979
(gen, order) = gen_order(twist_4)
s4 = [18979]
P41 = gen_subgroup(gen, order, s4[0])

# small subgroup for twist_6 are of order 10903, 5290657, 10833080827, 22921299619447
(gen, order) = gen_order(twist_6)
s6 = [10903, 5290657, 10833080827, 22921299619447]
P61 = gen_subgroup(gen, order, s6[0])
P62 = gen_subgroup(gen, order, s6[1])
P63 = gen_subgroup(gen, order, s6[2])
P64 = gen_subgroup(gen, order, s6[3])

# assert product of subgroup orders > 2^256

CPU times: user 47.8 s, sys: 235 ms, total: 48 s
Wall time: 47.9 s


In [6]:
%%time

# Replies received from Bob (with a naive implementation on Bob's side)
Q11 = b * P11
Q21 = b * P21
Q22 = b * P22
Q31 = b * P31
Q32 = b * P32
Q33 = b * P33
Q41 = b * P41
Q61 = b * P61
Q62 = b * P62
Q63 = b * P63
Q64 = b * P64

# computing 
x11 = discrete_log_rho(Q11, P11, ord=s1[0], operation='+')

x21 = discrete_log_rho(Q21, P21, ord=s2[0], operation='+')
x22 = discrete_log_rho(Q22, P22, ord=s2[1], operation='+')

x31 = discrete_log_rho(Q31, P31, ord=s3[0], operation='+')
x32 = discrete_log_rho(Q32, P32, ord=s3[1], operation='+')
x33 = discrete_log_rho(Q33, P33, ord=s3[2], operation='+')

x41 = discrete_log_rho(Q41, P41, ord=s4[0], operation='+')

x61 = discrete_log_rho(Q61, P61, ord=s6[0], operation='+')
x62 = discrete_log_rho(Q62, P62, ord=s6[1], operation='+')
x63 = discrete_log_rho(Q63, P63, ord=s6[2], operation='+')
x64 = discrete_log_rho(Q64, P64, ord=s6[3], operation='+')

x = crt([x11, x21, x22, x31, x32, x33, x41, x61, x62, x63, x64], [s1[0], s2[0], s2[1], s3[0], s3[1], s3[2], s4[0], s6[0], s6[1], s6[2], s6[3]])

print("Bob's private key is : ", x)
end_time = time.time()

print(end_time - start_time)

Bob's private key is :  141716600055626156383984504933026115190
925.3762125968933
CPU times: user 14min 36s, sys: 400 ms, total: 14min 37s
Wall time: 14min 37s
